In [16]:
import telegram
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import pandas as pd
import pandahouse
from read_db.CH import Getch
import os

sns.set()


def report(chat=None):
    """ Данная функция создает запрос в БД и присылает отчет по базовым метрикам в telegram"""
    chat_id = chat or 2093549033
    # bot = telegram.Bot(token=os.environ.get("REPORT_BOT_TOKEN"))  # шировка токена по пути Settings -> CI/CD -> Variables
    bot = telegram.Bot(token='*********************************************')
    
    query = '''
    select toStartOfDay(time) as day,
    count(DISTINCT user_id) as DAU,
    countIf(user_id, action = 'view') as views,
    countIf(user_id, action = 'like') as likes,
    countIf(user_id, action = 'like') / countIf(user_id, action = 'view') as CTR
    from {db}.feed_actions 
    where day <= yesterday() and day > yesterday() - 7
    group by day
    order by day desc
    '''
    #  FROM simulator_20220120.feed_actions

    
    # Создаем датафрейм 
    data = Getch(query).df
    
    # Отправка базовых метрик за вчерашний день
    msg = '''Отчет за {day}:\nDAU: {DAU}\nviews: {views}\nlikes: {likes}\nCTR: {CTR:.2%}'''.format(day=str(data.day[0]).split(' ')[0], 
                                                                                           DAU=data.DAU[0], 
                                                                                           views=data.views[0],
                                                                                           likes=data.likes[0],
                                                                                           CTR = data.CTR[0])
    
    bot.sendMessage(chat_id=chat_id, text= msg)
    
    
    # Отправка недельных графиков по базовым метрикам
    n = 1
    for metric in data.columns[1:]:
        y = data.iloc[:7, n]
        x = data.iloc[:7, 0]
        plt.title(metric)
        plt.xticks(rotation=15)
        plt.plot(x,y,'go-')
        n += 1

        plot_object = io.BytesIO()
        plt.savefig(plot_object)
        plot_object.name = 'plot.png'
        plot_object.seek(0)
        bot.sendPhoto(chat_id = chat_id, photo = plot_object)
        plt.close()



#     data = Getch('select * from simulator.feed_actions where toDate(time) = today() limit 100').df
#     file_object = io.StringIO()
#     data.to_csv(file_object)
#     file_object.name = 'test_file.csv'
#     file_object.seek(0)
#     bot.sendDocument(chat_id=chat_id, document=file_object)


try:
    report()
except Exception as e:
    print(e)